In [6]:
from flask import Flask
import eventlet.wsgi
import eventlet
import socketio
import numpy as np

import base64
from PIL import Image
from io import BytesIO

import matplotlib.pyplot as plt

from keras.models import load_model

In [ ]:
!ls

In [7]:
sio = socketio.Server()
app = Flask(__name__)

def send_control(steering_angle, throttle):
    sio.emit("steer", data={'steering_angle': str(steering_angle),
                            'throttle': str(throttle) }, skip_sid=True)

In [4]:
model = load_model('../day05/mymodel.h5')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalize (Lambda)           (None, 60, 80, 3)         0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 58, 78, 64)        1792      
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 56, 76, 32)        18464     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 28, 38, 32)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 26, 36, 16)        4624      
_______________________________________

In [ ]:
def process_image(img):
    return img[ 10:130:2, ::4, : ]

@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        
        speed = float(data["speed"])
        image_str = data["image"]

        decoded = base64.b64decode(image_str)
        image = Image.open(BytesIO(decoded))
        image_array = np.asarray(image)
        
#        plt.imshow(image_array)
#        plt.show()
        
#        print(image_array.shape)

        img = process_image(image_array)
        img_batch = np.expand_dims(img, axis=0)
        
        steering_angle = float(model.predict(img_batch))
                           
#        steering_angle = -1
        throttle = 0.15
    
        if speed < 15:
            throttle = 0.5
        if speed > 17:
            throttle = -0.1
    

        send_control(steering_angle, throttle)
    else:
        sio.emit('manual', data={}, skip_sid=True)
        
app = socketio.Middleware(sio, app)
eventlet.wsgi.server(eventlet.listen(('', 4567)), app)


(39451) wsgi starting up on http://0.0.0.0:4567
(39451) accepted ('127.0.0.1', 61807)
(39451) accepted ('127.0.0.1', 61981)
127.0.0.1 - - [17/May/2019 11:17:50] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 206.179813
(39451) accepted ('127.0.0.1', 62116)
127.0.0.1 - - [17/May/2019 11:19:57] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 183.558644
127.0.0.1 - - [17/May/2019 11:22:11] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 189.214745
